In [2]:
import qiskit
import scipy as sp
import pandas as pd
import numpy as np
from numpy import pi
import math as mt
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer, IBMQ
from qiskit.circuit import Clbit, Qubit
from qiskit.providers.aer.noise import NoiseModel
from random_swap_benchmark import *
from main import *

In [3]:
IBMQ.enable_account('74320ebeb744a83499d1ed5a7016fd513e98bb8550c495c13cee748e16b5f516f75b443de47bcdc96b8f9685b75a4023ba78a588ed34887ab526e019f3db4b9c')
provider = IBMQ.get_provider(
    hub='ibm-q-research-2',
    group='oxford-uni-1',
    project='main')

In [5]:
rsb = RandomSwapBenchmark(6,6, provider)
benchmark = rsb.randomized_benchmark(seed = 123, no_coord_samples = 10, no_state_samples = 10, mitigate = False)

In [17]:

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)

display(benchmark_df)

,x_i,y_i,x_j,y_j,x_b,y_b,KL bubble,KL regular,state seed,postselected,mitigated,KL bubble mean,KL regular mean,KL bubble error,KL regular error
0,1,5,2,3,4,2,5.756e-04,2.459e-04,0,1,0,7.261e-04,4.689e-04,7.281e-05,7.924e-05
1,1,5,2,3,4,2,6.877e-04,1.419e-04,1,1,0,7.261e-04,4.689e-04,7.281e-05,7.924e-05
2,1,5,2,3,4,2,1.049e-03,6.676e-04,2,1,0,7.261e-04,4.689e-04,7.281e-05,7.924e-05
3,1,5,2,3,4,2,5.289e-04,1.025e-03,3,1,0,7.261e-04,4.689e-04,7.281e-05,7.924e-05
4,1,5,2,3,4,2,1.002e-03,3.748e-04,4,1,0,7.261e-04,4.689e-04,7.281e-05,7.924e-05
5,1,5,2,3,4,2,1.025e-03,3.011e-04,5,1,0,7.261e-04,4.689e-04,7.281e-05,7.924e-05
6,1,5,2,3,4,2,5.658e-04,3.601e-04,6,1,0,7.261e-04,4.689e-04,7.281e-05,7.924e-05
7,1,5,2,3,4,2,8.465e-04,3.690e-04,7,1,0,7.261e-04,4.689e-04,7.281e-05,7.924e-05
8,1,5,2,3,4,2,6.426e-04,4.826e-04,8,1,0,7.261e-04,4.689e-04,7.281e-05,7.924e-05
9,1,5,2,3,4,2,3.380e-04,7.204e-04,9,1,0,7.261e-04,4.689e-04,7.281e-05,7.924e-05


In [16]:
kl_avg_bubble = np.zeros(100)
kl_avg_regular = np.zeros(100)
kl_se_bubble = np.zeros(100)
kl_se_regular = np.zeros(100)
for i in range(10):
    b = np.mean(benchmark_df['KL bubble'][10*i:10*(i+1)])
    r = np.mean(benchmark_df['KL regular'][10*i:10*(i+1)])
    b_se = np.std(benchmark_df['KL bubble'][10*i:10*(i+1)]) / np.sqrt(10)
    r_se = np.std(benchmark_df['KL regular'][10*i:10*(i+1)]) / np.sqrt(10)
    kl_avg_bubble[10*i:10*(i+1)] = b
    kl_avg_regular[10*i:10*(i+1)] = r
    kl_se_bubble[10*i:10*(i+1)] = b_se
    kl_se_regular[10*i:10*(i+1)] = r_se
benchmark_df['KL bubble mean'] = kl_avg_bubble
benchmark_df['KL regular mean'] = kl_avg_regular
benchmark_df['KL bubble error'] = kl_se_bubble
benchmark_df['KL regular error'] = kl_se_regular
    

In [4]:
rsb = RandomSwapBenchmark(4,2, provider)
res = []
for j in range(10):
    res.append(RandomSwapBenchmark(4,2,provider).routing_test(0, 0, 3, 0, 0, 1, 3, 1, postselect = True, mitigate = False, seed = j))
res_df = pd.DataFrame(res)


bubble 1 at [0, 1]
bubble 2 at [3, 1]
bubble 1 at [0, 1]
bubble 2 at [3, 1]
bubble 1 at [0, 1]
bubble 2 at [3, 1]
bubble 1 at [0, 1]
bubble 2 at [3, 1]
bubble 1 at [0, 1]
bubble 2 at [3, 1]
bubble 1 at [0, 1]
bubble 2 at [3, 1]
bubble 1 at [0, 1]
bubble 2 at [3, 1]
bubble 1 at [0, 1]
bubble 2 at [3, 1]
bubble 1 at [0, 1]
bubble 2 at [3, 1]
bubble 1 at [0, 1]
bubble 2 at [3, 1]
bubble 1 at [0, 1]
bubble 2 at [3, 1]
bubble 1 at [0, 1]
bubble 2 at [3, 1]
bubble 1 at [0, 1]
bubble 2 at [3, 1]
bubble 1 at [0, 1]
bubble 2 at [3, 1]
bubble 1 at [0, 1]
bubble 2 at [3, 1]
bubble 1 at [0, 1]
bubble 2 at [3, 1]
bubble 1 at [0, 1]
bubble 2 at [3, 1]
bubble 1 at [0, 1]
bubble 2 at [3, 1]
bubble 1 at [0, 1]
bubble 2 at [3, 1]
bubble 1 at [0, 1]
bubble 2 at [3, 1]


In [5]:
display(res_df)

,x_i,y_i,x_j,y_j,x_b1,y_b1,x_b2,y_b2,KL bubble,KL regular,KL double bubble,state seed,postselected,mitigated
0,0,0,3,0,0,1,3,1,0.001492,0.002316,0.001333,0,1,0
1,0,0,3,0,0,1,3,1,0.003047,0.000609,0.001857,1,1,0
2,0,0,3,0,0,1,3,1,0.029384,0.014308,0.033562,2,1,0
3,0,0,3,0,0,1,3,1,0.005746,0.004692,0.014192,3,1,0
4,0,0,3,0,0,1,3,1,0.050305,0.028261,0.041312,4,1,0
5,0,0,3,0,0,1,3,1,0.004230,0.002399,0.007490,5,1,0
6,0,0,3,0,0,1,3,1,0.009926,0.007080,0.010126,6,1,0
7,0,0,3,0,0,1,3,1,0.014337,0.010692,0.017796,7,1,0
8,0,0,3,0,0,1,3,1,0.008592,0.003604,0.002978,8,1,0
9,0,0,3,0,0,1,3,1,0.207801,0.166333,0.270321,9,1,0


In [25]:
kl_bubble = np.array([np.mean(res_df['KL bubble'])]*10)
kl_regular = np.array([np.mean(res_df['KL regular'])]*10)
res_df['kl bubble mean'] = kl_bubble
res_df['kl regular mean'] = kl_regular

In [31]:
benchmark_df.to_csv('data/6x6_single_bubble_benchmark.csv')

In [3]:
rsb = RandomSwapBenchmark(6,2, provider, print_statements = True)
dbsc = rsb.DoubleBubbleSwapCircuit(provider, 6,2, 0,0,5,0,0,1,5,1, print_statements = True)
dbsc.push_i()

bubble 1 at [0, 1]
bubble target at [1, 0], data target at [0, 0]
distance = 2
moved bubble 1 from [0, 1] to [1, 1]
distance = 1
moved bubble 1 from [1, 1] to [1, 0]
distance = 0
swapped I at [0, 0] with bubble 1 at [1, 0]
bubble 2 at [5, 1]
bubble target at [4, 0], data target at [5, 0]
distance = 2
moved bubble from [5, 1] to [4, 1]
distance = 1
moved bubble from [4, 1] to [4, 0]
distance = 0
swapped J at [5, 0] with bubble 2 at [4, 0]
bubble 1 at [0, 0]
bubble target at [2, 0], data target at [1, 0]
distance = 4
moved bubble 1 from [0, 0] to [0, 1]
distance = 3
moved bubble 1 from [0, 1] to [1, 1]
distance = 2
moved bubble 1 from [1, 1] to [2, 1]
distance = 1
moved bubble 1 from [2, 1] to [2, 0]
distance = 0
swapped I at [1, 0] with bubble 1 at [2, 0]
bubble 2 at [5, 0]
bubble target at [3, 0], data target at [4, 0]
distance = 4
moved bubble from [5, 0] to [5, 1]
distance = 3
moved bubble from [5, 1] to [4, 1]
distance = 2
moved bubble from [4, 1] to [3, 1]
distance = 1
moved bubble

In [12]:
rsb = RandomSwapBenchmark(4,2, provider, print_statements = True)
dbsc = rsb.BubbleSwapCircuit(provider, 4,2, 0,0,3,0,0,1)
dbsc.push_states_together()
rsc = rsb.RegularSwapCircuit(provider, 4, 2, 0,0,3,0, 0,1,dbsc.swap_path)

In [15]:
rsc.push_states_together()
rsc.gcirc.circuit.draw()

┌──────────────────────────┐      
q9985_0: ┤ U(2.188,0.89893,0.71267) ├─X────
         └┬────────────────────────┬┘ │    
q9985_1: ─┤ U(1.732,2.2603,1.3292) ├──X──X─
         ┌┴────────────────────────┤     │ 
q9985_2: ┤ U(3.0812,2.1515,1.5109) ├─────X─
         ├─────────────────────────┤       
q9985_3: ┤ U(1.2319,1.0781,2.2904) ├───────
         └─────────────────────────┘       
q9985_4: ──────────────────────────────────
         ┌──────────────────────────┐      
q9985_5: ┤ U(2.3185,0.57331,0.5512) ├──────
         └┬────────────────────────┬┘      
q9985_6: ─┤ U(1.6699,1.6708,1.993) ├───────
         ┌┴────────────────────────┤       
q9985_7: ┤ U(2.6686,2.2759,1.9196) ├───────
         └─────────────────────────┘       
 c170: 2/══════════════════════════════════
                                           
c171: 36/══════════════════════════════════

In [20]:
print([dbsc.x_i, dbsc.y_i, dbsc.x_j, dbsc.y_j, dbsc.x_b, dbsc.y_b])


[2, 0, 3, 0, 1, 0]
